In [36]:
from keras.models import load_model
import os
import cv2
import numpy as np

In [37]:
model = load_model('handgesmodel.h5')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)             

In [38]:
path = 'gestureimgs'
imgs=[]
for i in (os.listdir(path)):
    img_path = os.path.join(path,i)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (500,500))
    imgs.append(img)

In [39]:
code = np.arange(0,3)
label = dict(zip(code,imgs))

In [40]:
cap = cv2.VideoCapture(0)
x,y,w,h = 300,50,350,350
while cap.isOpened():
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    mask2 = cv2.inRange(hsv, np.array([60,55,200]), np.array([120,110,255]))
    res = cv2.bitwise_and(frame,frame,mask=mask2)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    median = cv2.GaussianBlur(gray, (5,5), 0)
      
    kernel_square = np.ones((5,5), np.uint8)
    dilation = cv2.dilate(median, kernel_square, iterations=2)
    opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
      
    ret,thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
    thresh = thresh[y:y+h, x:x+w]
    contours = cv2.findContours(thresh.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.imshow("Capturing gesture...",frame)
    
    if len(contours)>0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 2500:
            x1, y1, w1, h1 = cv2.boundingRect(contour)
            final_img = thresh[y1:y1+h, x1:x1+w1]
            final_img = cv2.resize(final_img,(50,50))
            final_img = np.reshape(final_img, (1,50,50,1))
            out = model.predict(final_img)
            output=np.argmax(out,axis=1)[0]
            cv2.imshow('THRESH',thresh)
            cv2.imshow('Output',label[output])
            
    key=cv2.waitKey(1)
    if key==27:
        break

cv2.destroyAllWindows()
cap.release()